# Author:

- Huu Khang Nguyen - 7402909
- hkn878@uowmail.edu.au


# Environment:

- Python 3.10.6
- Ubuntu 22.04.2 LTS x86_64

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
# columns data from http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names
columns = [
    "duration",
    "protocol_type",
    "service",
    "flag",
    "src_bytes",
    "dst_bytes",
    "land",
    "wrong_fragment",
    "urgent",
    "hot",
    "num_failed_logins",
    "logged_in",
    "num_compromised",
    "root_shell",
    "su_attempted",
    "num_root",
    "num_file_creations",
    "num_shells",
    "num_access_files",
    "num_outbound_cmds",
    "is_host_login",
    "is_guest_login",
    "count",
    "srv_count",
    "serror_rate",
    "srv_serror_rate",
    "rerror_rate",
    "srv_rerror_rate",
    "same_srv_rate",
    "diff_srv_rate",
    "srv_diff_host_rate",
    "dst_host_count",
    "dst_host_srv_count",
    "dst_host_same_srv_rate",
    "dst_host_diff_srv_rate",
    "dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate",
    "dst_host_serror_rate",
    "dst_host_srv_serror_rate",
    "dst_host_rerror_rate",
    "dst_host_srv_rerror_rate",
    "target"
]

df = pd.read_csv('./kddcup.data_10_percent_corrected', names=columns)


In [4]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [5]:
df.isna().sum()

duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_h

In [6]:
df.dtypes

duration                         int64
protocol_type                   object
service                         object
flag                            object
src_bytes                        int64
dst_bytes                        int64
land                             int64
wrong_fragment                   int64
urgent                           int64
hot                              int64
num_failed_logins                int64
logged_in                        int64
num_compromised                  int64
root_shell                       int64
su_attempted                     int64
num_root                         int64
num_file_creations               int64
num_shells                       int64
num_access_files                 int64
num_outbound_cmds                int64
is_host_login                    int64
is_guest_login                   int64
count                            int64
srv_count                        int64
serror_rate                    float64
srv_serror_rate          

In [7]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

In [8]:
categorical_cols

['protocol_type', 'service', 'flag', 'target']

In [9]:
df['target'].value_counts()

target
smurf.              280790
neptune.            107201
normal.              97278
back.                 2203
satan.                1589
ipsweep.              1247
portsweep.            1040
warezclient.          1020
teardrop.              979
pod.                   264
nmap.                  231
guess_passwd.           53
buffer_overflow.        30
land.                   21
warezmaster.            20
imap.                   12
rootkit.                10
loadmodule.              9
ftp_write.               8
multihop.                7
phf.                     4
perl.                    3
spy.                     2
Name: count, dtype: int64

I realised that there's way too many target label to predict, so therefore I will map these label according to http://kdd.ics.uci.edu/databases/kddcup99/training_attack_types so that we can make the model run faster

In [10]:
df['target'] = df['target'].replace({
    'normal.': 'normal',
    'back.': 'dos',
    'buffer_overflow.': 'u2r',
    'ftp_write.': 'r2l',
    'guess_passwd.': 'r2l',
    'imap.': 'r2l',
    'ipsweep.': 'probe',
    'land.': 'dos',
    'loadmodule.': 'u2r',
    'multihop.': 'r2l',
    'neptune.': 'dos',
    'nmap.': 'probe',
    'perl.': 'u2r',
    'phf.': 'r2l',
    'pod.': 'dos',
    'portsweep.': 'probe',
    'rootkit.': 'u2r',
    'satan.': 'probe',
    'smurf.': 'dos',
    'spy.': 'r2l',
    'teardrop.': 'dos',
    'warezclient.': 'r2l',
    'warezmaster.': 'r2l'
})


In [11]:
df['target'].value_counts()

target
dos       391458
normal     97278
probe       4107
r2l         1126
u2r           52
Name: count, dtype: int64

Label encoding all the categorical columns for Gaussian Naive Bayes 

In [12]:
for col in categorical_cols:
    label_dict = {}
    for indx, label in enumerate(df[col].unique()):
        label_dict[label] = indx
    df[col] = df[col].map(label_dict)

In [13]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target
0,0,0,0,0,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,0
1,0,0,0,0,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0
2,0,0,0,0,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0
3,0,0,0,0,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0
4,0,0,0,0,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0


## Implement Naive Bayes from scratch

In [14]:
import numpy as np

In [35]:
class GaussianNaiveBayes:
    def __init__(self, smoothing=1e-9):
        self.classes = []
        self.class_priors = {} # prior probability
        self.means = {} # mean of each feature for each class
        self.variances = {} # standard deviation of each feature for each class
        self.smoothing = smoothing # laplace smoothing
        
    def fit(self, X, y):
        self.classes = np.unique(y)
        for class_ in self.classes:
            match_rows = X[y == class_]
            self.class_priors[class_] = len(match_rows) + self.smoothing / len(X) + self.smoothing 
            self.means[class_] = np.mean(match_rows, axis=0)
            self.variances[class_] = np.var(match_rows, axis=0)

    def gaussian_pdf(self, x, mean, variance):
        exponent = np.exp(-((x - mean) ** 2) / (2 * variance))
        return (1 / np.sqrt(2 * np.pi * variance)) * exponent

    def predict(self, X):
        y_pred = []

        for _, x in X.iterrows():
            posteriors = []

            for class_ in self.classes:
                # Bayes theorem
                prior = self.class_priors[class_]
                likelihood = self.gaussian_pdf(x, self.means[class_], self.variances[class_])

                # posterior probability
                posterior = prior * np.prod(likelihood)
                posteriors.append(posterior)

            predicted_class = self.classes[np.argmax(posteriors)]
            y_pred.append(predicted_class)

        return y_pred

## Split the dataset to 70% training and 30% testing

In [23]:
from sklearn.naive_bayes import GaussianNB

In [24]:
train = df.sample(frac=0.7, random_state=42)
test = df.drop(train.index)

In [32]:
y_train = train['target']
X_train = train.drop('target', axis=1)

y_test = test['target']
X_test = test.drop('target', axis=1)

In [37]:
nb = GaussianNaiveBayes()
nb.fit(X_train, y_train)

In [38]:
y_pred = nb.predict(X_test)

In [39]:
accuracy = np.mean(y_pred == y_test)

In [40]:
accuracy

0.9178778187117931